# <span style="color: rgb(138, 92, 245);">I. Encuesta en Aerolíneas</span>

## <span style="color: rgb(138, 92, 245);">1. Introducción</span>

Comenzamos por importar los datos referentes a este ejercicio. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Tema oscuro por defecto
px.defaults.template = "plotly_dark"

In [2]:
df_1 = pd.read_csv('data/data_practica_encuesta.csv')

In [3]:
df_1.head()

,Id,Servicio wifi a bordo,Hora de salida/llegada conveniente,Facilidad de reserva en línea,Ubicación de la puerta,Alimentos y bebidas,Embarque en línea,Comodidad del asiento,Entretenimiento a bordo,Servicio a bordo,Servicio de sala de piernas,Manejo de equipaje,Servicio de facturación,Limpieza
0,19556,5,4,3,4,3,4,3,5,5,5,5,2,5
1,90035,1,1,3,1,5,4,5,4,4,4,4,3,5
2,12360,2,0,2,4,2,2,2,2,4,1,3,2,2
3,77959,0,0,0,2,3,4,4,1,1,1,1,3,4
4,36875,2,3,4,3,4,1,2,2,2,2,2,4,4


In [4]:
df_1.shape

(25976, 14)

El presente ejercicio tiene como objetivo analizar la opinión de los usuarios de una aerolínea sobre diversos aspectos del servicio. A partir de los datos proporcionados, se evaluará la calidad de los datos, se realizará un análisis exploratorio y se aplicará un Análisis Factorial para reducir la dimensionalidad y obtener componentes latentes que representen los factores subyacentes en la percepción de los usuarios.

## <span style="color: rgb(138, 92, 245);">2. Calidad de los datos</span>

**Objetivo**: validar si los datos son consistentes, completos y adecuados para el análisis.
Incluye:

* Verificación de valores nulos o atípicos.

* Revisión de tipos de datos.

* Identificación de valores extremos o inconsistentes.

 * Decisiones de limpieza o imputación.

In [5]:
# Funcion Para Imprimir Porcetanje de Valores Nulos y Tipos de Datos
def print_data_info(df):
    print("Información del DataFrame:")
    print(df.info())
    print("\n Porcentaje de Valores nulos por columna:")
    print(df.isnull().mean() * 100)
    print("\n Cantidad de Valores Únicos por columna:")
    for column in df.columns:
        unique_count = df[column].value_counts()
        print(f"{column}: {unique_count} valores únicos")

In [6]:
# Excluimmos la primera columna que es un ID
df_1_original = df_1.copy()
df_1 = df_1.iloc[:,1:]

In [7]:
# Imprimir Información del DataFrame y Porcentaje de Valores Nulos (Excluyendo la primera columna)
print_data_info(df_1)

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   Servicio wifi a bordo               25976 non-null  int64
 1   Hora de salida/llegada conveniente  25976 non-null  int64
 2   Facilidad de reserva en línea       25976 non-null  int64
 3   Ubicación de la puerta              25976 non-null  int64
 4   Alimentos y bebidas                 25976 non-null  int64
 5   Embarque en línea                   25976 non-null  int64
 6   Comodidad del asiento               25976 non-null  int64
 7   Entretenimiento a bordo             25976 non-null  int64
 8   Servicio a bordo                    25976 non-null  int64
 9   Servicio de sala de piernas         25976 non-null  int64
 10  Manejo de equipaje                  25976 non-null  int64
 11  Servicio de facturación             2597

Los resultados anteriores nos indican que toda variable es numérica por lo que se consideraron aptas para el Análisis Factorial. Además, ninguna variable contiene valores nulos explícitamente. Tampoco se tienen variables ***dummy***, pues ninguna es de tipo binaria (tampoco unaria).

También podemos notar que todas las variables toman valores enteros del 1 al 5, es decir, existen 5 tipos de calificaciones distintas que pueden elegir los clientes, con 1 la más baja y 5 la más alta. En este caso, según nuestro diccionario, el valor cero indica que no se proporcionó calificación. Por lo cuál, el valor cero represtaria un valor nulo en este caso. Procedemos a revisar la cantidad de ceros en las variables, luego, si su porcentaje de ceros es menor al 10% los transformamos a valores `np.nan` y con ello, podremos imputar los valores nulos. 

In [8]:
# Función para contar ceros en el DataFrame
def zeros_percentage(df):
    for column in df.columns:
        zero_count = (df[column] == 0).sum()
        total_count = len(df[column])
        zero_percentage = (zero_count / total_count) * 100
        print(f"Columna '{column}': {zero_count} ceros ({zero_percentage:.2f}%)")   

# Función para reemplazar ceros con NaN
def fill_zero_with_nan(df):
    df_replaced = df.replace(0, np.nan)
    return df_replaced

In [9]:
# Mostrar estadisticas descriptivas (como la media) antes de imputar
df_1.describe()

,Servicio wifi a bordo,Hora de salida/llegada conveniente,Facilidad de reserva en línea,Ubicación de la puerta,Alimentos y bebidas,Embarque en línea,Comodidad del asiento,Entretenimiento a bordo,Servicio a bordo,Servicio de sala de piernas,Manejo de equipaje,Servicio de facturación,Limpieza
count,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000,25976.000000
mean,2.724746,3.046812,2.756775,2.977094,3.215353,3.261665,3.449222,3.357753,3.385664,3.350169,3.633238,3.314175,3.286226
std,1.335384,1.533371,1.412951,1.282133,1.331506,1.355536,1.320090,1.338299,1.282088,1.318862,1.176525,1.269332,1.319330
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,2.000000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,3.000000
75%,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [10]:
# Revisar porcentaje de ceros en el DataFrame
zeros_percentage(df_1)

Columna 'Servicio wifi a bordo': 813 ceros (3.13%)
Columna 'Hora de salida/llegada conveniente': 1381 ceros (5.32%)
Columna 'Facilidad de reserva en línea': 1195 ceros (4.60%)
Columna 'Ubicación de la puerta': 0 ceros (0.00%)
Columna 'Alimentos y bebidas': 25 ceros (0.10%)
Columna 'Embarque en línea': 652 ceros (2.51%)
Columna 'Comodidad del asiento': 0 ceros (0.00%)
Columna 'Entretenimiento a bordo': 4 ceros (0.02%)
Columna 'Servicio a bordo': 2 ceros (0.01%)
Columna 'Servicio de sala de piernas': 126 ceros (0.49%)
Columna 'Manejo de equipaje': 0 ceros (0.00%)
Columna 'Servicio de facturación': 0 ceros (0.00%)
Columna 'Limpieza': 2 ceros (0.01%)


Dado que el porcentaje de ceros en todas las columnas es menor al 10%, podemos remplazarlos por valores nulos e imputarlos por la moda, pues necesitamos que los valores imputados sean también valores enteros. 

In [11]:
# Reemplazar ceros con NaN
df_1 = fill_zero_with_nan(df_1)

In [12]:
from sklearn.impute import SimpleImputer

# Función para imputar valores nulos con la moda
def impute_missing_with_mode(df):
    # Definir el imputador para la moda
    imputer = SimpleImputer(strategy='most_frequent')
    for column in df.columns:
        df[column] = imputer.fit_transform(df[[column]]).astype(int)

    return df

In [13]:
# Imputar valores nulos con la moda
df_1 = impute_missing_with_mode(df_1)

In [14]:
print_data_info(df_1)

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   Servicio wifi a bordo               25976 non-null  int64
 1   Hora de salida/llegada conveniente  25976 non-null  int64
 2   Facilidad de reserva en línea       25976 non-null  int64
 3   Ubicación de la puerta              25976 non-null  int64
 4   Alimentos y bebidas                 25976 non-null  int64
 5   Embarque en línea                   25976 non-null  int64
 6   Comodidad del asiento               25976 non-null  int64
 7   Entretenimiento a bordo             25976 non-null  int64
 8   Servicio a bordo                    25976 non-null  int64
 9   Servicio de sala de piernas         25976 non-null  int64
 10  Manejo de equipaje                  25976 non-null  int64
 11  Servicio de facturación             2597

Otro punto importante que debemos revisar es verificar que existen correlaciones entre las variables que tenemos.

In [15]:
# Heatmap de correlación
corr = df_1.corr()

fig_corr = px.imshow(corr, text_auto=True, aspect="auto",
                     title="Matriz de Correlación (numéricas)",
                     color_continuous_scale="Purp")
fig_corr.show()

Es claro que distintas variables presentan correlaciones positivas entre ellas. Por otro lado, revisamos con el metodo de Z-Score si existen valores extremos que necesitemos tratar. 

In [16]:
from scipy import stats

# --- Calcular Z-score ---
z_scores = np.abs(stats.zscore(df_1))

# --- Identificar filas con algún valor extremo ---
outliers_z = (z_scores > 3)   # umbral clásico
outlier_rows = df_1[outliers_z.any(axis=1)]

print(f"Filas con valores extremos detectadas: {len(outlier_rows)}")

Filas con valores extremos detectadas: 0


En este caso, no se detectan valores extremos o atípicos que representen un problema para nuestro Análisis Factorial. 

## <span style="color: rgb(138, 92, 245);">3. Escalamiento</span>

Sabemos que, para el Análisis Factorial, necesitaremos hacer posteriormente dos tipos de pruebas: 
* Prueba de Esfericidad de Bartlett
* Prueba KMO

Ambas pruebas se basan en la matriz de correlación que graficamos anteriormente. Por ello, aunque las variables no tengan escalas tan distintas, es recomendable hacer un escalamiento de datos estandar (media $\mu=0$ y desviación estandar $\sigma=1$)

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
scaler =  StandardScaler()
columns=df_1.columns
df_1 = scaler.fit_transform(df_1)
df_1 = pd.DataFrame(data=df_1,columns=columns)
df_1.head(10)

,Servicio wifi a bordo,Hora de salida/llegada conveniente,Facilidad de reserva en línea,Ubicación de la puerta,Alimentos y bebidas,Embarque en línea,Comodidad del asiento,Entretenimiento a bordo,Servicio a bordo,Servicio de sala de piernas,Manejo de equipaje,Servicio de facturación,Limpieza
0,1.769652,0.542883,0.117204,0.797831,-0.165067,0.508483,-0.340303,1.227251,1.259257,1.255453,1.161716,-1.035348,1.299060
1,-1.429490,-1.656419,0.117204,-1.542065,1.341006,0.508483,1.174774,0.479671,0.479062,0.485439,0.311739,-0.247517,1.299060
2,-0.629705,0.542883,-0.657870,0.797831,-0.918104,-1.085670,-1.097842,-1.015491,0.479062,-1.824605,-0.538238,-1.035348,-0.975382
3,-0.629705,0.542883,-0.657870,-0.762100,-0.165067,0.508483,0.417235,-1.763071,-1.861523,-1.824605,-2.238193,-0.247517,0.540913
4,-0.629705,-0.190218,0.892278,0.017866,0.587969,-1.882746,-1.097842,-1.015491,-1.081328,-1.054590,-1.388216,0.540315,0.540913
5,0.170081,-0.190218,0.117204,0.017866,1.341006,1.305559,-0.340303,1.227251,0.479062,-0.284576,-2.238193,-1.823179,1.299060
6,1.769652,1.275983,1.667352,1.577797,-0.165067,1.305559,1.174774,1.227251,1.259257,1.255453,1.161716,0.540315,-0.217235
7,-0.629705,-0.923318,-0.657870,-0.762100,0.587969,0.508483,1.174774,0.479671,0.479062,0.485439,0.311739,1.328146,-0.217235
8,1.769652,-0.923318,-0.657870,-0.762100,1.341006,1.305559,1.174774,1.227251,-1.081328,-1.054590,1.161716,-0.247517,1.299060
9,-0.629705,-0.923318,-0.657870,-0.762100,-0.165067,0.508483,0.417235,0.479671,0.479062,0.485439,0.311739,1.328146,0.540913


## <span style="color: rgb(138, 92, 245);">4. Prueba de Esfericidad de Bartlett</span>

La siguiente prueba propone como hipótesis nula que la matriz de correlación sea igual a la matriz identidad de su dimensión correspondiente. El proposito es rechazar la hipótesis nula con un nivel de siginificancia $\alpha=0.05$, pues esto nos diría que sí existen correlaciones entre nuestras variables. 

$$H_0: C_{n\times n}=\text{Id}_n\,\hspace{5mm}\text{vs}\hspace{5mm} H_1: C_{n\times n}\neq\text{Id}_n $$
donde $c_{ij}=\text{Corr}(X_i,X_j)=\frac{\text{Cov}(X_i,X_j)}{\sigma_{X_i}\sigma_{X_j}}$ para toda $i,j\in\{1,2,\dots,n\}$.

In [19]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

In [20]:
# Prueba de Esfericidad de Bartlett
chi2,p = calculate_bartlett_sphericity(df_1)
print("Esfericidad de Bartlett")
print("Valor de Chi : ",chi2)
print("P - value : ",p)

Esfericidad de Bartlett
Valor de Chi :  133504.84001600655
P - value :  0.0


Dado que $p-\text{value}=0.0<0.05=\alpha$, entonces rechazamos la hipótesis nula, es decir, existen variables correlacionadas con un 95% de confianza. 

## <span style="color: rgb(138, 92, 245);">5. Prueba KMO</span>

Esta prueba nos permite comprobar que existe varianza común entre variables y que cada variable puede predecirse por las demás variables dentro del dataset. En este caso, se considera que un $KMO<0.6$ sería inadecuado para el Análisis Factorial, pues indicaría que la proporción de varianza en general es de menos del 60%. 

In [21]:
from factor_analyzer.factor_analyzer import calculate_kmo

In [22]:
# Prueba KMO
kmo_all,kmo_model = calculate_kmo(df_1)
print("KMO Test Statisitc",kmo_model)

KMO Test Statisitc 0.7624124186131424


In [23]:
kmo_all

array([0.74558039, 0.734667  , 0.70489763, 0.72060627, 0.83744217,
       0.73320624, 0.8321386 , 0.74327996, 0.71314235, 0.82656594,
       0.72706304, 0.66170088, 0.81927677])

Se obtuvo una proporción general de varianza del 76%, lo cuál nos indica que tiene sentido proceder a reducir dimensiones con el Análisis Factorial. 

## <span style="color: rgb(138, 92, 245);">6. Número de Factores</span>

Dado que ya se ha eliminado la columna *Id* de nuestro análisis, tomamos todas las variables restantes y procedemos a obtener los factores más significativos, es decir, aquellos con mayor información respcto a la varianza común entre variables.

In [24]:
from factor_analyzer import FactorAnalyzer

In [25]:
fa = FactorAnalyzer(rotation = None)

In [26]:
fa.fit(df_1)

c:\Users\Aura De La Garza G\Projects\CienciaDeDatos_M3\venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



,n_factors,3
,rotation,None
,method,'minres'
,use_smc,True
,is_corr_matrix,False
,bounds,"(0.005, ...)"
,impute,'median'
,svd_method,'randomized'
,rotation_kwargs,{}


Para poder obtener los factores más sifgnificativos, necesitamos calcular los eigenvalores de la matriz de correlación. Aquellos eigenvalores que sean mayores a 1, serán los más aptos para el análisis. Esto es porque, al escalar nuestros datos, su varianza (medida por sus eigenvalores) al ser mayor que 1, nos indica que es una variable que explica más varianza que una sola variable observada. 

In [27]:
ev,_ = fa.get_eigenvalues()
# Mostrar eigenvalores
print("Eigenvalores:", np.round(ev, 3))

Eigenvalores: [3.592 2.438 1.794 1.038 0.934 0.668 0.503 0.482 0.453 0.334 0.291 0.28
 0.192]


Creamos una gráfica interactiva que nos permita ver tanto el valor del eigenvalor del factor como su porcentaje de varianza explicada (Llamado Gráfico de Scree). 

In [28]:
import plotly.graph_objects as go
import pandas as pd

# Calcular porcentaje de varianza explicada
var_exp = [(i / sum(ev)) * 100 for i in ev]

# Crear DataFrame
df_ev = pd.DataFrame({
    'Factor': range(1, len(ev)+1),
    'Eigenvalor': ev,
    'VarianzaExplicada(%)': var_exp
})

# Crear figura base
fig = go.Figure()

# Añadir puntos y línea punteada azul pastel
fig.add_trace(go.Scatter(
    x=df_ev['Factor'],
    y=df_ev['Eigenvalor'],
    mode='lines+markers+text',
    text=[f"λ={v:.2f}<br>{p:.1f}%" for v, p in zip(df_ev['Eigenvalor'], df_ev['VarianzaExplicada(%)'])],
    textposition='top center',
    line=dict(color='#89CFF0', width=2, dash='dot'),
    marker=dict(color='#89CFF0', size=8),
    name='Eigenvalores',
    hovertemplate="<b>Factor %{x}</b><br>Eigenvalor: %{y:.2f}<br>Varianza explicada: %{customdata:.1f}%<extra></extra>",
    customdata=df_ev['VarianzaExplicada(%)']
))

# Personalizar estilo del hover (fondo negro, texto blanco)
fig.update_traces(
    hoverlabel=dict(
        bgcolor='black', 
        font_size=13,
        font_color='white'
    )
)

# Añadir línea horizontal del criterio de Kaiser
fig.add_hline(
    y=1,
    line_dash="dash",
    line_color="gray",
    annotation_text="Eigenvalor = 1 (criterio de Kaiser)",
    annotation_position="bottom right",
    annotation_font_color="gray"
)

# Configurar layout general
fig.update_layout(
    template='plotly_dark',
    title='Gráfico de Scree (Eigenvalores y Varianza Explicada por Factor)',
    title_font=dict(size=22, color='white'),
    xaxis_title='Factores',
    yaxis_title='Eigenvalores',
    hovermode='x unified',
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    width=1250,
    height=750,
)

# Cuadrícula gris suave
fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='gray')
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='gray')

# Mostrar
fig.show()

Notamos que los primeros cuatro factores estan por encima de 1. Por lo tanto, el número de factores que elegimos serán 4. 

## <span style="color: rgb(138, 92, 245);">7. Interpretación de Factores</span>

In [29]:
fa = FactorAnalyzer(n_factors=4,rotation=None)
fa.fit(df_1)

c:\Users\Aura De La Garza G\Projects\CienciaDeDatos_M3\venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



,n_factors,4
,rotation,None
,method,'minres'
,use_smc,True
,is_corr_matrix,False
,bounds,"(0.005, ...)"
,impute,'median'
,svd_method,'randomized'
,rotation_kwargs,{}


In [30]:
fa = FactorAnalyzer(n_factors=4,rotation='varimax')
fa.fit(df_1)

c:\Users\Aura De La Garza G\Projects\CienciaDeDatos_M3\venv\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



,n_factors,4
,rotation,'varimax'
,method,'minres'
,use_smc,True
,is_corr_matrix,False
,bounds,"(0.005, ...)"
,impute,'median'
,svd_method,'randomized'
,rotation_kwargs,{}


In [31]:
pd.DataFrame(fa.get_eigenvalues()[0],columns=['EigenValues'])

,EigenValues
0,3.591788
1,2.438426
2,1.794445
3,1.038265
4,0.933908
5,0.667518
6,0.502751
7,0.482149
8,0.453045
9,0.334337


## <span style="color: rgb(138, 92, 245);">8. Cargas</span>

In [32]:
import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)

In [33]:
fa.loadings_

array([[ 0.11030726,  0.64617799,  0.1354356 ,  0.36993264],
       [-0.04127085,  0.61941502,  0.0492302 , -0.0306301 ],
       [-0.02214046,  0.8169483 ,  0.02359169,  0.29975442],
       [ 0.01491591,  0.69534145, -0.05429057, -0.14332475],
       [ 0.77751112,  0.01868742, -0.00360688,  0.04291785],
       [ 0.27397035,  0.14626108,  0.1409472 ,  0.80284405],
       [ 0.75022034, -0.01829127,  0.0826572 ,  0.2354729 ],
       [ 0.78298106,  0.03129913,  0.46180855,  0.0070922 ],
       [ 0.07283772,  0.0199196 ,  0.72854787,  0.03628884],
       [ 0.06253361,  0.04363083,  0.50257134,  0.06038175],
       [ 0.04817686,  0.04148012,  0.73558996, -0.03422526],
       [ 0.0901516 , -0.01421404,  0.29055015,  0.17376996],
       [ 0.84696326, -0.0053772 ,  0.08598628,  0.11900872]])

In [34]:
cargas=pd.DataFrame(fa.loadings_,index=df_1.columns).style.background_gradient(cmap=cm)

cargas

,0,1,2,3
Servicio wifi a bordo,0.110307,0.646178,0.135436,0.369933
Hora de salida/llegada conveniente,-0.041271,0.619415,0.049230,-0.030630
Facilidad de reserva en línea,-0.022140,0.816948,0.023592,0.299754
Ubicación de la puerta,0.014916,0.695341,-0.054291,-0.143325
Alimentos y bebidas,0.777511,0.018687,-0.003607,0.042918
Embarque en línea,0.273970,0.146261,0.140947,0.802844
Comodidad del asiento,0.750220,-0.018291,0.082657,0.235473
Entretenimiento a bordo,0.782981,0.031299,0.461809,0.007092
Servicio a bordo,0.072838,0.019920,0.728548,0.036289
Servicio de sala de piernas,0.062534,0.043631,0.502571,0.060382


## <span style="color: rgb(138, 92, 245);">9. Diferencia</span>

In [39]:
fa.get_factor_variance()

(array([2.60706773, 1.97943293, 1.68051449, 1.00057463]),
 array([0.20054367, 0.15226407, 0.12927035, 0.07696728]),
 array([0.20054367, 0.35280774, 0.48207809, 0.55904537]))

In [40]:
diferencia=pd.DataFrame(fa.get_factor_variance(),index=['Variance','Proportional Var','Cumulative Var'])

diferencia

,0,1,2,3
Variance,2.607068,1.979433,1.680514,1.000575
Proportional Var,0.200544,0.152264,0.129270,0.076967
Cumulative Var,0.200544,0.352808,0.482078,0.559045


## <span style="color: rgb(138, 92, 245);">10. Comunidades</span>

In [44]:
comunidades=pd.DataFrame(fa.get_uniquenesses(),index=df_1.columns,columns=['unicidad'])

comunidades

,unicidad
Servicio wifi a bordo,0.415093
Hora de salida/llegada conveniente,0.611260
Facilidad de reserva en línea,0.241696
Ubicación de la puerta,0.492788
Alimentos y bebidas,0.393272
Embarque en línea,0.239123
Comodidad del asiento,0.374555
Entretenimiento a bordo,0.172644
Servicio a bordo,0.462199
Servicio de sala de piernas,0.737962


# <span style="color: rgb(138, 92, 245);">II. FNA de Masa Mamaria</span>

In [35]:
df_2 = pd.read_csv('data/data_practica_cancer.csv')

In [36]:
df_2.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# <span style="color: rgb(138, 92, 245);">III. Lenguage de Señas</span>

In [37]:
df_3 = pd.read_csv('data/data_practica_lenguaje_señas.csv')

In [38]:
df_3.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,...,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,...,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,...,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,...,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,...,26,40,64,48,29,46,49,46,46,53
